In [1]:
!pip install -U unsloth unsloth_zoo bitsandbytes accelerate transformers peft sentence-transformers scipy xformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 146.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954

In [2]:
!pip uninstall torch torchaudio torchvision -y


Found existing installation: torch 2.9.1
Uninstalling torch-2.9.1:
  Successfully uninstalled torch-2.9.1
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: torchvision 0.24.1
Uninstalling torchvision-0.24.1:
  Successfully uninstalled torchvision-0.24.1


In [3]:
!pip install torch==2.9.1+cu128 torchvision==0.24.1+cu128 torchaudio==2.9.1+cu128 --index-url https://download.pytorch.org/whl/cu128


Looking in indexes: https://download.pytorch.org/whl/cu128
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 900.9/900.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.5 MB/s eta 0:00:00


In [4]:
import torch
import torchaudio
print(torch.__version__, torchaudio.__version__)
print(torch.__version__, torch.version.cuda, torch.cuda.is_available())

from unsloth import FastLanguageModel
print("Unsloth import succeeded")


2.9.1+cu128 2.9.1+cu128
2.9.1+cu128 12.8 True
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth import succeeded


In [5]:
from datasets import Dataset
import numpy as np
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [6]:
# Check GPU
!nvidia-smi


Mon Dec 29 09:43:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P0             26W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import gc, torch, os

gc.collect()
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [8]:


import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import KLDivLoss
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, PreTrainedModel
from scipy.stats import spearmanr
from tqdm import tqdm
import os

print("✅ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


✅ All imports successful!
PyTorch version: 2.9.1+cu128
CUDA available: True


In [9]:
def make_soft_labels_empirical(choices):
    """Use ACTUAL human rating distribution instead of Gaussian"""
    counts = np.zeros(5, dtype=np.float32)
    for rating in choices:
        counts[rating - 1] += 1
    probs = counts / counts.sum()
    return probs

def make_soft_labels_adaptive(avg, stdev, choices):
    """Adaptive smoothing based on agreement level"""
    centers = np.arange(1, 6, dtype=np.float32)
    num_ratings = len(choices)
    unique_ratings = len(set(choices))
    if unique_ratings == 1:
        sigma = 0.3
    elif unique_ratings == 2 and stdev < 1.0:
        sigma = 0.5
    else:
        sigma = min(max(stdev, 0.4), 1.5)
    dist = np.exp(-0.5 * ((centers - avg) / sigma)**2)
    return dist / dist.sum()

def mark_target_word(text, homonym):
    """Add [TGT] markers around the homonym"""
    return text.replace(homonym, f"[TGT] {homonym}")

print("✅ Helper functions defined!")


✅ Helper functions defined!


In [10]:
def load_json_file(file_path):
    """
    Load either strict JSON or line-delimited JSON objects
    """
    data_list = []
    with open(file_path, "r") as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char in ["{", "["]:
            # normal JSON
            data = json.load(f)
            data_list = list(data.values()) if isinstance(data, dict) else data
        else:
            # line-delimited or Python-style
            import ast
            for line in f:
                line = line.strip()
                if line:
                    data_list.append(ast.literal_eval(line))
    return data_list

TRAIN_FILE = "/content/combined_train.json"
DEV_FILE = "/content/dev.json"

train_list = load_json_file(TRAIN_FILE)
dev_list = load_json_file(DEV_FILE)

print(f"📊 Loaded {len(train_list)} training samples")
print(f"📊 Loaded {len(dev_list)} dev samples")


📊 Loaded 7448 training samples
📊 Loaded 588 dev samples


In [11]:
def check_types(data_list):
    type_map = {}
    for i, item in enumerate(data_list):
        for key, value in item.items():
            if key not in type_map:
                type_map[key] = {}
            val_type = type(value).__name__
            if val_type not in type_map[key]:
                type_map[key][val_type] = []
            type_map[key][val_type].append(i)
    print("\n🔍 Checking for mixed types:")
    for key, types in type_map.items():
        if len(types) > 1:
            print(f"\n❌ Field '{key}' has mixed types:")
            for t, indices in types.items():
                print(f"   {t}: {len(indices)} occurrences (first at index {indices[0]})")
                if indices[0] < len(data_list):
                    print(f"      Example value: {data_list[indices[0]][key]}")

check_types(train_list)

def clean_data(data_list):
    """Ensure all fields have consistent types for Arrow"""
    cleaned = []
    for item in data_list:
        cleaned.append({
            'id': int(item.get('id',0)) if item.get('id') is not None else 0,
            'homonym': str(item.get('homonym','')),
            'judged_meaning': str(item.get('judged_meaning','')),
            'precontext': str(item.get('precontext') or ''),
            'sentence': str(item.get('sentence','')),
            'ending': str(item.get('ending') or ''),
            'choices': list(item.get('choices', [])) if isinstance(item.get('choices'), list) else [],
            'nonsensical': bool(item.get('nonsensical', False)),
            'average': float(item.get('average',0)) if item.get('average') is not None else 0.0,
            'stdev': float(item.get('stdev',0.0)) if item.get('stdev') is not None else 0.0,
            'example_sentence': str(item.get('example_sentence') or '')
        })
    return cleaned

train_list = clean_data(train_list)
dev_list = clean_data(dev_list)
print("✅ Data cleaned!")



🔍 Checking for mixed types:

❌ Field 'average' has mixed types:
   float: 5024 occurrences (first at index 0)
      Example value: 3.0
   int: 2424 occurrences (first at index 2281)
      Example value: 5
✅ Data cleaned!


In [12]:
def preprocess_ambistory(row):
    story = (
        row["precontext"].strip() + " " +
        row["sentence"].strip() + " " +
        row["ending"].strip()
    ).strip()
    story = mark_target_word(story, row["homonym"])

    if row["example_sentence"] and len(str(row["example_sentence"]).strip()) > 0:
        example = f"Definition: {row['judged_meaning']}. Example: {row['example_sentence']}"
    else:
        example = f"Definition: {row['judged_meaning']}"

    labels = make_soft_labels_empirical(row["choices"])
    return {
        "story": story,
        "example": example,
        "labels": labels,
        "stdev": row["stdev"]
    }

from datasets import Dataset

train_ds = Dataset.from_list(train_list)
dev_ds = Dataset.from_list(dev_list)

train_ds = train_ds.map(preprocess_ambistory)
dev_ds = dev_ds.map(preprocess_ambistory)

print("✅ Preprocessing complete!")
print(f"\nExample preprocessed sample:")
print(f"  Story (first 100 chars): {train_ds[0]['story'][:100]}...")
print(f"  Example (first 100 chars): {train_ds[0]['example'][:100]}...")
print(f"  Labels: {train_ds[0]['labels']}")


Map:   0%|          | 0/7448 [00:00<?, ? examples/s]

Map:   0%|          | 0/588 [00:00<?, ? examples/s]

✅ Preprocessing complete!

Example preprocessed sample:
  Story (first 100 chars): The old machine hummed in the corner of the workshop. Clara examined its dusty dials with a furrowed...
  Example (first 100 chars): Definition: the difference in electrical charge between two points in a circuit expressed in volts. ...
  Labels: [0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224]


In [28]:
from unsloth import FastLanguageModel

# # model_name = "unsloth/llama-3-8b-bnb-4bit"
# model_name = "unsloth/mistral-7b-v0.3-bnb-4bit"

# base_model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name=model_name,
#     max_seq_length=2048,
#     load_in_4bit=True,
#     device_map="auto",
#     offload_folder="offload",
#     output_hidden_states=True  # <-- important!,
#     precision="fp16"
# )

# # Attach LoRA adapters
# base_model = FastLanguageModel.get_peft_model(
#     base_model,
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.0,
#     bias="none",
#     target_modules=[
#         "q_proj","k_proj","v_proj","o_proj",
#         "gate_proj","up_proj","down_proj",
#     ],
# )
model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=512,
    load_in_4bit=True,
    device_map="auto",
    output_hidden_states=True,
    torch_dtype="bfloat16",
)

base_model = FastLanguageModel.get_peft_model(
    base_model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.0,
    target_modules=["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj"],
)

base_model.gradient_checkpointing_disable()
print("✅ LLaMA + LoRA loaded.")



==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ LLaMA + LoRA loaded.


In [29]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import KLDivLoss
from transformers import PreTrainedModel

class SoftLLaMAModel(PreTrainedModel):
    config_class = base_model.config_class

    def __init__(self, config, base_model):
        super().__init__(config)
        self.backbone = base_model
        hidden = config.hidden_size
        self.classifier = nn.Linear(hidden, 5)
        self.kl_loss = KLDivLoss(reduction="batchmean")

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # Use hidden_states instead of last_hidden_state
        hidden_states = outputs.hidden_states[-1]  # last layer
        pooled = hidden_states.mean(dim=1)        # mean pooling over sequence
        logits = self.classifier(pooled)
        # hidden = outputs.hidden_states.mean(dim=1)
        # logits = self.classifier(hidden)

        if labels is None:
            return {"logits": logits}

        log_probs = torch.log_softmax(logits, dim=-1)
        kl_loss = self.kl_loss(log_probs, labels)

        probs = torch.softmax(logits, dim=-1)
        centers = torch.tensor([1,2,3,4,5], dtype=torch.float32, device=logits.device)
        pred_score = (probs * centers).sum(dim=-1)
        true_score = (labels * centers).sum(dim=-1)
        mse_loss = F.mse_loss(pred_score, true_score)

        loss = 0.7 * kl_loss + 0.3 * mse_loss
        return {"loss": loss, "logits": logits, "kl_loss": kl_loss, "mse_loss": mse_loss}


model = SoftLLaMAModel(base_model.config, base_model)
model.to(device)
print("✅ Soft-label LLaMA model initialized")


✅ Soft-label LLaMA model initialized


In [17]:
def tokenize_fn(row):
    enc = tokenizer(
        row["story"],
        row["example"],
        truncation=True,
        padding="max_length",
        max_length=128,  # smaller seq length saves memory
        return_tensors="pt"
    )
    enc["labels"] = torch.tensor(row["labels"], dtype=torch.float32)
    return enc

train_tok = train_ds.map(tokenize_fn)
dev_tok = dev_ds.map(tokenize_fn)

train_tok.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
dev_tok.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print("✅ Tokenization complete!")
print(f"Train samples: {len(train_tok)}, Dev samples: {len(dev_tok)}")


Map:   0%|          | 0/7448 [00:00<?, ? examples/s]

Map:   0%|          | 0/588 [00:00<?, ? examples/s]

✅ Tokenization complete!
Train samples: 7448, Dev samples: 588


In [31]:
from scipy.stats import spearmanr
import numpy as np

# def compute_metrics(eval_pred):
#     # Unpack values safely (Unsloth may wrap logits)
#     logits, labels = eval_pred

#     # Convert logits to a single 2-D array
#     if isinstance(logits, tuple):
#         logits = logits[0]

#     # If logits are a list of tensors → stack
#     if isinstance(logits, list):
#         logits = np.vstack([np.array(x) for x in logits])
#     else:
#         logits = np.array(logits)

#     # Same for labels
#     if isinstance(labels, list):
#         labels = np.vstack([np.array(x) for x in labels])
#     else:
#         labels = np.array(labels)

#     # Convert logits → probabilities
#     logits_exp = np.exp(logits - np.max(logits, axis=-1, keepdims=True))
#     probs = logits_exp / np.sum(logits_exp, axis=-1, keepdims=True)

#     centers = np.array([1, 2, 3, 4, 5])
#     pred_scores = np.sum(probs * centers, axis=1)
#     true_scores = np.sum(labels * centers, axis=1)

#     spear = spearmanr(pred_scores, true_scores).correlation
#     acc_1 = np.mean(np.abs(pred_scores - true_scores) <= 1.0)

#     return {
#         "spearman": float(spear),
#         "accuracy_within_sd": float(acc_1),
#     }
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert to numpy first (they're already numpy arrays from Trainer)
    if isinstance(logits, tuple):
        logits = logits[0]

    logits = np.array(logits)
    labels = np.array(labels)

    # Compute probabilities
    # Apply softmax manually
    logits_exp = np.exp(logits - np.max(logits, axis=-1, keepdims=True))
    probs = logits_exp / np.sum(logits_exp, axis=-1, keepdims=True)

    centers = np.array([1, 2, 3, 4, 5])

    # Compute predicted scores (expected value)
    pred_scores = np.sum(probs * centers, axis=1)

    # Compute true scores (expected value from soft labels)
    true_scores = np.sum(labels * centers, axis=1)

    # Spearman correlation
    spear = spearmanr(pred_scores, true_scores).correlation

    # Accuracy within stdev
    # Get stdevs from original dataset
    try:
        stdevs = np.array([dev_ds[i]["stdev"] for i in range(len(pred_scores))])
        stdevs = np.maximum(stdevs, 1.0)  # Minimum threshold of 1.0
        sd_acc = np.mean(np.abs(pred_scores - true_scores) <= stdevs)
    except:
        # Fallback if stdev not available
        sd_acc = np.mean(np.abs(pred_scores - true_scores) <= 1.0)

    # Also compute fixed ±1.0 accuracy
    acc_1 = np.mean(np.abs(pred_scores - true_scores) <= 1.0)

    return {
        "spearman": float(spear),
        "sd_accuracy": float(sd_acc),
        "accuracy_1": float(acc_1)
    }


print("✅ Metrics function ready")


✅ Metrics defined (FIXED)!
✅ Metrics function ready


In [19]:
train_tok.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
dev_tok.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [20]:
# Example for a single batch
batch = train_tok[0]
print(type(batch))
print(batch.keys() if isinstance(batch, dict) else batch)
print(batch['input_ids'].shape if 'input_ids' in batch else batch.shape)


<class 'dict'>
dict_keys(['labels', 'input_ids', 'attention_mask'])
torch.Size([1, 128])


In [21]:
for batch in train_tok:
    print(batch['input_ids'].shape)
    break


torch.Size([1, 128])


In [22]:
def flatten_batch(batch):
    batch['input_ids'] = batch['input_ids'].squeeze(0)
    batch['attention_mask'] = batch['attention_mask'].squeeze(0)
    batch['labels'] = batch['labels'].squeeze(0)
    return batch

train_tok = train_tok.map(flatten_batch)
dev_tok = dev_tok.map(flatten_batch)


Map:   0%|          | 0/7448 [00:00<?, ? examples/s]

Map:   0%|          | 0/588 [00:00<?, ? examples/s]

In [33]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./llama_output",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # keep 1 to fit in memory
    gradient_accumulation_steps=4,  # emulate batch size 4
    per_device_eval_batch_size=1,
    learning_rate=1e-5,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy_1",
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=dev_tok,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")


/tmp/ipython-input-2925598465.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,448 | Num Epochs = 3 | Total steps = 5,586
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 11,282,437 of 1,247,096,837 (0.90% trained)


🚀 Starting training...


Epoch,Training Loss,Validation Loss,Spearman,Sd Accuracy,Accuracy 1
1,0.698800,0.808441,0.495284,0.726190,0.675170
2,0.609300,0.796722,0.515195,0.738095,0.687075
3,0.533600,0.811679,0.514463,0.755102,0.687075


There were missing keys in the checkpoint model loaded: ['backbone.base_model.model.lm_head.weight'].
There were unexpected keys in the checkpoint model loaded: ['backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.nested_absmax', 'backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.nested_quant_map', 'backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.quant_map', 'backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.absmax', 'backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'backbone.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.nested_absmax', 'backbone.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.nested_quant_map', 'backbone.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.quant_map', 'backbone.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.absmax', '

✅ Training complete!


In [34]:
# ============================================================
# Save the best model
# ============================================================
MODEL_SAVE_DIR = "./best_llama_model"

print("💾 Saving best model...")
trainer.save_model(MODEL_SAVE_DIR)
tokenizer.save_pretrained(MODEL_SAVE_DIR)

# Save training info
training_info = {
    "model_name": model_name,
    "num_epochs": training_args.num_train_epochs,
    "learning_rate": training_args.learning_rate,
    "batch_size": training_args.per_device_train_batch_size,
    "modifications": [
        "Empirical soft labels from actual ratings",
        "Rich meaning representation (definition + example)",
        "Hybrid loss (KL + MSE)",
        "Adaptive smoothing based on agreement",
        "Optimized for accuracy metric"
    ]
}

with open(f"{MODEL_SAVE_DIR}/training_info.json", "w") as f:
    json.dump(training_info, f, indent=2)

print(f"✅ Model saved to: {MODEL_SAVE_DIR}")
print("\n📊 Final metrics on validation set:")

# Get final evaluation
final_metrics = trainer.evaluate()
for key, value in final_metrics.items():
    if key.startswith("eval_"):
        print(f"   {key}: {value:.4f}")

💾 Saving best model...
✅ Model saved to: ./best_llama_model

📊 Final metrics on validation set:


   eval_loss: 0.7967
   eval_spearman: 0.5152
   eval_sd_accuracy: 0.7381
   eval_accuracy_1: 0.6871
   eval_runtime: 46.5026
   eval_samples_per_second: 12.6440
   eval_steps_per_second: 12.6440


In [37]:

from safetensors.torch import load_file
print("🔮 Generating predictions...")

# Reload best model
model = SoftLLaMAModel(base_model.config, base_model)
state_dict = load_file(f"{MODEL_SAVE_DIR}/model.safetensors")
model.load_state_dict(state_dict)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

all_predictions = []

with torch.no_grad():
    for i in tqdm(range(len(dev_tok)), desc="Predicting"):
        sample = {
            "input_ids": dev_tok[i]["input_ids"].unsqueeze(0).to(model.device),
            "attention_mask": dev_tok[i]["attention_mask"].unsqueeze(0).to(model.device)
        }

        outputs = model(**sample)
        logits = outputs["logits"]
        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

        # Compute expected value
        centers = np.array([1, 2, 3, 4, 5])
        pred_score = (probs * centers).sum()
        pred_class = int(np.clip(np.round(pred_score), 1, 5))

        all_predictions.append(pred_class)

print(f"✅ Generated {len(all_predictions)} predictions")

🔮 Generating predictions...


RuntimeError: Error(s) in loading state_dict for SoftLLaMAModel:
	Missing key(s) in state_dict: "backbone.base_model.model.lm_head.weight". 
	Unexpected key(s) in state_dict: "backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.nested_absmax", "backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.nested_quant_map", "backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.quant_map", "backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.absmax", "backbone.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.quant_state.bitsandbytes__nf4", "backbone.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.nested_absmax", "backbone.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.nested_quant_map", "backbone.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.quant_map", "backbone.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weig

In [ ]:

import os

OUTPUT_DIR = "input/res"
os.makedirs(OUTPUT_DIR, exist_ok=True)

output_path = f"{OUTPUT_DIR}/predictions.jsonl"

with open(output_path, "w") as f:
    for idx, pred in enumerate(all_predictions):
        record = {"id": str(idx), "prediction": pred}
        f.write(json.dumps(record) + "\n")

print(f"✅ Predictions saved to: {output_path}")

# Show distribution of predictions
unique, counts = np.unique(all_predictions, return_counts=True)
print("\n📊 Prediction distribution:")
for score, count in zip(unique, counts):
    print(f"   Score {score}: {count} samples ({count/len(all_predictions)*100:.1f}%)")

# Download the file
print("\n📥 Downloading predictions file...")
from google.colab import files
files.download(output_path)

In [ ]:

# Upload solution file
print("Upload solution.jsonl if you want to evaluate:")
uploaded_sol = files.upload()

if "solution.jsonl" in uploaded_sol:
    os.makedirs("input/ref", exist_ok=True)
    os.rename("solution.jsonl", "input/ref/solution.jsonl")

    # Run scoring (assuming you have scoring.py)
    !python scoring.py input/ref/solution.jsonl input/res/predictions.jsonl output/scores.json

    # Load and display results
    if os.path.exists("output/scores.json"):
        with open("output/scores.json") as f:
            scores = json.load(f)

        print("\n" + "="*70)
        print("🏆 FINAL RESULTS")
        print("="*70)
        print(f"Spearman Correlation: {scores.get('spearman', 'N/A')}")
        print(f"Accuracy: {scores.get('accuracy', 'N/A')}")
        print("="*70)
else:
    print("⚠️ No solution file uploaded. Skipping evaluation.")

In [ ]:
!python3 scoring.py input/ref/solution.jsonl input/res/predictions.jsonl output/scores.json
!zip -j my_submission.zip input/res/predictions.jsonl

from google.colab import files
files.download("my_submission.zip")


In [ ]:
MODEL_SAVE_DIR = "./best_llama_model"
trainer.save_model(MODEL_SAVE_DIR)
tokenizer.save_pretrained(MODEL_SAVE_DIR)
print(f"✅ Model saved at {MODEL_SAVE_DIR}")


In [38]:
model.eval()
all_predictions = []

with torch.no_grad():
    for i in tqdm(range(len(dev_tok))):
        sample = {k: dev_tok[i][k].unsqueeze(0).to(device) for k in ["input_ids","attention_mask"]}
        outputs = model(**sample)
        logits = outputs["logits"]
        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
        centers = np.array([1,2,3,4,5])
        pred_score = (probs * centers).sum()
        pred_class = int(np.clip(np.round(pred_score), 1, 5))
        all_predictions.append(pred_class)

print(f"✅ Predictions generated: {len(all_predictions)}")


  0%|          | 0/588 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but got mat1 is on cuda:0, different from other tensors on cpu (when checking argument in method wrapper_CUDA_addmm)